Abra esse arquivo no colab do google. Coloque a sua chave no Secrets: GOOGLE_API_KEY e depois aperte CTRL F9

A ideia do projeto é receber mensagens através do Telegram e usar bots para gerar arquivos ou mensagens para a IA distribuir os produtos de doações. Os armazéns mandam mensagens informando seu estoque, os abrigos mandam os dados do local, e também as suas necessidades. O Python irá encaminhar as mensagens para a IA do Google, que distribuirá os itens conforme a necessidade de cada abrigo, e também gerará uma lista em forma de mensagem que será enviada de volta aos armazéns para que a distribuição seja feita. Também daria para colocar no projeto a IA analisando as mensagens recebidas via bot do Telegram e ela gerando as listas de estoque ou de necessidades (ela interpretaria o que as pessoas escreveram). Eu não sei implementar a parte do Telegram; quem souber e puder ajudar, seria muito bem-vindo. Mais ideias poderão ser anexadas ao projeto para ajudar o povo do Sul ou de outras localidades.

In [26]:
!pip install -q -U google.generativeai

In [27]:
import google.generativeai as genai
from google.colab import userdata
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

In [28]:
generation_config = {
    'candidate_count':1,
    'temperature':0.5,
    'top_p':0.5,
}

safety_settings = {
    'HARASSMENT':'BLOCK_NONE',
    'HATE':'BLOCK_NONE',
    'SEXUAL':'BLOCK_NONE',
    'DANGEROUS':'BLOCK_NONE',
}

In [29]:
model = genai.GenerativeModel('gemini-1.0-pro',
                              generation_config=generation_config,
                              safety_settings=safety_settings,
                              )

As listas abaixo podem ser textos recebidos por bot do telegram, ou serem arquivos textos ou em excel. Coloquei em formato de listas para ipynb ler mais facil sem usar arquivos externos. Só executar com o CTRL F9.

In [30]:
# Lista de abrigos e contatos
abrigos = [
    {"Nome": "Casa Esperança", "Endereço": "Rua da Solidariedade, 123, Cidade Nova, Estado X", "Telefone": "(XX) XXXX-XXXX", "Email": "casaesperanca@example.com", "Representante": "Ana Silva", "Camas Totais": 50, "Vagas Livres": 12},
    {"Nome": "Lar Renascer", "Endereço": "Avenida da União, 456, Bairro Feliz, Estado Y", "Telefone": "(YY) YYYY-YYYY", "Email": "larrenascer@example.com", "Representante": "João Santos", "Camas Totais": 40, "Vagas Livres": 8},
    {"Nome": "Abrigo Amizade", "Endereço": "Travessa da Fraternidade, 789, Vila Acolhedora, Estado Z", "Telefone": "(ZZ) ZZZZ-ZZZZ", "Email": "abrigoamizade@example.com", "Representante": "Maria Oliveira", "Camas Totais": 60, "Vagas Livres": 20},
    {"Nome": "Residência Solidária", "Endereço": "Praça da Compaixão, 321, Aldeia da Esperança, Estado W", "Telefone": "(WW) WWWW-WWWW", "Email": "residenciasolidaria@example.com", "Representante": "Carlos Mendes", "Camas Totais": 55, "Vagas Livres": 15},
    {"Nome": "Lar Fraterno", "Endereço": "Rua da Paz, 654, Vila União, Estado V", "Telefone": "(VV) VVVV-VVVV", "Email": "larfraterno@example.com", "Representante": "Ana Paula Costa", "Camas Totais": 48, "Vagas Livres": 10},
    {"Nome": "Abrigo União Familiar", "Endereço": "Avenida da Esperança, 987, Bairro Harmonia, Estado U", "Telefone": "(UU) UUUU-UUUU", "Email": "abrigouniaofamiliar@example.com", "Representante": "Pedro Alves", "Camas Totais": 70, "Vagas Livres": 25},
    {"Nome": "Casa da Solidariedade", "Endereço": "Rua da Amizade, 234, Comunidade Unidos, Estado T", "Telefone": "(TT) TTTT-TTTT", "Email": "casadasolidariedade@example.com", "Representante": "Renata Oliveira", "Camas Totais": 45, "Vagas Livres": 5}
]


In [31]:
solicitacoes = [
    "Casa Esperança: Nós da Casa Esperança precisamos de 63 litros de leite, 76 unidades de fraldas, 72 unidades de cobertores, 92 unidades de toalhas de banho, 34 unidades de pratos, 71 unidades de talheres, 31 unidades de panelas, 52 unidades de sacos de lixo, 74 unidades de sabonetes, 61 unidades de pastas de dentes, 60 unidades de escovas de dentes",
    "Lar Renascer: Nós do Lar Renascer precisamos de 23 litros de leite, 53 unidades de fraldas, 95 unidades de cobertores, 21 unidades de toalhas de banho, 89 unidades de pratos, 89 unidades de talheres, 94 unidades de panelas, 51 unidades de sacos de lixo, 21 unidades de sabonetes, 47 unidades de pastas de dentes, 91 unidades de escovas de dentes",
    "Abrigo Amizade: Nós do Abrigo Amizade precisamos de 50 litros de leite, 33 unidades de fraldas, 67 unidades de cobertores, 47 unidades de toalhas de banho, 28 unidades de pratos, 84 unidades de talheres, 74 unidades de panelas, 40 unidades de sacos de lixo, 82 unidades de sabonetes, 33 unidades de pastas de dentes, 51 unidades de escovas de dentes",
    "Residência Solidária: Nós da Residência Solidária precisamos de 39 litros de leite, 26 unidades de fraldas, 31 unidades de cobertores, 33 unidades de toalhas de banho, 61 unidades de pratos, 50 unidades de talheres, 57 unidades de panelas, 14 unidades de sacos de lixo, 79 unidades de sabonetes, 20 unidades de pastas de dentes, 94 unidades de escovas de dentes",
    "Lar Fraterno: Nós do Lar Fraterno precisamos de 65 litros de leite, 48 unidades de fraldas, 21 unidades de cobertores, 58 unidades de toalhas de banho, 43 unidades de pratos, 94 unidades de talheres, 92 unidades de panelas, 14 unidades de sacos de lixo, 45 unidades de sabonetes, 16 unidades de pastas de dentes, 79 unidades de escovas de dentes",
    "Abrigo União Familiar: Nós do Abrigo União Familiar precisamos de 17 litros de leite, 26 unidades de fraldas, 29 unidades de cobertores, 70 unidades de toalhas de banho, 60 unidades de pratos, 61 unidades de talheres, 26 unidades de panelas, 68 unidades de sacos de lixo, 73 unidades de sabonetes, 23 unidades de pastas de dentes, 61 unidades de escovas de dentes",
    "Casa da Solidariedade: Nós da Casa da Solidariedade precisamos de 22 litros de leite, 78 unidades de fraldas, 97 unidades de cobertores, 80 unidades de toalhas de banho, 83 unidades de pratos, 82 unidades de talheres, 28 unidades de panelas, 19 unidades de sacos de lixo, 83 unidades de sabonetes, 77 unidades de pastas de dentes, 78 unidades de escovas de dentes"
]


In [32]:
armazens = [
    "Armazém da Esperança: No Armazém da Esperança temos 2637 litros de leite, 2042 unidades de fraldas, 3120 unidades de cobertores, 1798 unidades de toalhas de banho, 4201 unidades de pratos, 3925 unidades de talheres, 2289 unidades de panelas, 3354 unidades de sacos de lixo, 1457 unidades de sabonetes, 2966 unidades de pastas de dentes, 4008 unidades de escovas de dentes",
    "Depósito Solidário: No Depósito Solidário temos 4893 litros de leite, 1376 unidades de fraldas, 3978 unidades de cobertores, 2731 unidades de toalhas de banho, 3712 unidades de pratos, 4032 unidades de talheres, 4875 unidades de panelas, 2389 unidades de sacos de lixo, 3985 unidades de sabonetes, 4721 unidades de pastas de dentes, 2286 unidades de escovas de dentes",
    "Estoque Fraterno: No Estoque Fraterno temos 2894 litros de leite, 4863 unidades de fraldas, 2549 unidades de cobertores, 3847 unidades de toalhas de banho, 2156 unidades de pratos, 4298 unidades de talheres, 2793 unidades de panelas, 1764 unidades de sacos de lixo, 4753 unidades de sabonetes, 3629 unidades de pastas de dentes, 4532 unidades de escovas de dentes",
    "Galpão da União: No Galpão da União temos 3761 litros de leite, 3892 unidades de fraldas, 4327 unidades de cobertores, 2156 unidades de toalhas de banho, 4876 unidades de pratos, 3174 unidades de talheres, 4825 unidades de panelas, 3971 unidades de sacos de lixo, 2569 unidades de sabonetes, 3085 unidades de pastas de dentes, 2096 unidades de escovas de dentes",
    "Depósito Amigo: No Depósito Amigo temos 2527 litros de leite, 4993 unidades de fraldas, 3498 unidades de cobertores, 2891 unidades de toalhas de banho, 4097 unidades de pratos, 2746 unidades de talheres, 3157 unidades de panelas, 4261 unidades de sacos de lixo, 3689 unidades de sabonetes, 2945 unidades de pastas de dentes, 4310 unidades de escovas de dentes",
    "Centro de Doações Felizes: No Centro de Doações Felizes temos 4085 litros de leite, 2837 unidades de fraldas, 4859 unidades de cobertores, 2264 unidades de toalhas de banho, 3972 unidades de pratos, 3096 unidades de talheres, 4950 unidades de panelas, 2213 unidades de sacos de lixo, 3927 unidades de sabonetes, 3298 unidades de pastas de dentes, 4932 unidades de escovas de dentes",
    "Armazém da Solidariedade: No Armazém da Solidariedade temos 3456 litros de leite, 2689 unidades de fraldas, 4632 unidades de cobertores, 3281 unidades de toalhas de banho, 4520 unidades de pratos, 2543 unidades de talheres, 3756 unidades de panelas, 4213 unidades de sacos de lixo, 2938 unidades de sabonetes, 3827 unidades de pastas de dentes, 2874 unidades de escovas de dentes",
    "Estoque Generoso: No Estoque Generoso temos 3156 litros de leite, 4098 unidades de fraldas, 2198 unidades de cobertores, 3947 unidades de toalhas de banho, 3067 unidades de pratos, 2468 unidades de talheres, 4378 unidades de panelas, 4819 unidades de sacos de lixo, 2539 unidades de sabonetes, 4483 unidades de pastas de dentes, 3075 unidades de escovas de dentes",
    "Depósito do Bem: No Depósito do Bem temos 3774 litros de leite, 4872 unidades de fraldas, 3961 unidades de cobertores, 2756 unidades de toalhas de banho, 4635 unidades de pratos, 2998 unidades de talheres, 3496 unidades de panelas, 4035 unidades de sacos de lixo, 3227 unidades de sabonetes, 4547 unidades de pastas de dentes, 3189 unidades de escovas de dentes",
    "Galpão da Alegria: No Galpão da Alegria temos 2926 litros de leite, 3875 unidades de fraldas, 4756 unidades de cobertores, 3412 unidades de toalhas de banho, 2593 unidades de pratos, 4376 unidades de talheres, 4982 unidades de panelas, 3021 unidades de sacos de lixo, 4815 unidades de sabonetes, 3729 unidades de pastas de dentes, 4185 unidades de escovas de dentes"
]


In [34]:
response = model.generate_content(f"""
Utilizando os dados dos armazéns onde os produtos a serem doados estão armazenados,
localize as necessidades dos abrigos e faça a distribuição dos produtos conforme a demanda.
Em seguida, gere mensagens individuais em forma de frase que serão enviadas via Telegram
para cada armazém, informando as listas de produtos, quantidades e os dados dos abrigos que
receberão os produtos, para o carregamento dos caminhões de entrega.
A mensagem também deve conter os dados completos dos abrigos para onde irá o caminhoneiro.
abrigos:{abrigos}
solicitacoes:{solicitacoes}
armazens{armazens}
""")
print(response.text)

**Distribuição de Produtos para Abrigos**

**Armazém da Esperança**

* **Abrigo:** Casa Esperança
* **Endereço:** Rua da Solidariedade, 123, Cidade Nova, Estado X
* **Produtos:**
    * Leite: 63 litros
    * Fraldas: 76 unidades
    * Cobertores: 72 unidades
    * Toalhas de banho: 92 unidades
    * Pratos: 34 unidades
    * Talheres: 71 unidades
    * Panelas: 31 unidades
    * Sacos de lixo: 52 unidades
    * Sabonetes: 74 unidades
    * Pastas de dentes: 61 unidades
    * Escovas de dentes: 60 unidades

**Mensagem para o Armazém da Esperança:**

Carregue o caminhão com os seguintes produtos para entrega à Casa Esperança:
* Leite: 63 litros
* Fraldas: 76 unidades
* Cobertores: 72 unidades
* Toalhas de banho: 92 unidades
* Pratos: 34 unidades
* Talheres: 71 unidades
* Panelas: 31 unidades
* Sacos de lixo: 52 unidades
* Sabonetes: 74 unidades
* Pastas de dentes: 61 unidades
* Escovas de dentes: 60 unidades

**Endereço de entrega:**
Rua da Solidariedade, 123
Cidade Nova, Estado X

**Dep

Um sistema mais complexo poderia ser incrementado,  obtendo também a localização do armazém e do abrigo fazendo o calculo para ver a distância. E enviar os caminhões mais próximos. Com isso melhorando a logística.